<a href="https://colab.research.google.com/github/SunWenlu/-/blob/master/AutoEncoder_UNetBased_LiDAR_Data_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1st round of training a (Variational)AutoEncoder on LiDAR statistics**
Date: 20/06/2022
Wenlu Sun

In [ ]:
# 1
# Load the sample data from personal Google Drive

from google.colab import drive
drive.mount('/content/drive')
import os

Project_Path = "/content/drive/MyDrive/TUM/"
# sample1_Path is regraded as training data
sample1_Path = "/content/drive/MyDrive/TUM/sam-1-1647360000_19571765_4ee45dac0acf65d7db1dac9aebb54bc3/"
# sample1_Path is regraded as testing data
sample2_Path = "/content/drive/MyDrive/TUM/sam-2-1648137600_35424916_c2e49754aa5b577c06c6d685ee45e7a6/"
os.chdir(sample1_Path)

In [ ]:
# 2
import keras
from PIL import Image
from skimage import data, io, filters
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from sklearn import manifold, datasets
import glob
from keras.preprocessing import image
import gdal
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate
import torch
import torchvision
from keras.callbacks import TensorBoard

In [ ]:
# 3
# Multi channel images loading ## 多channel图像读取
# Read image pixel matrix ## 读取图像像素矩阵
# fileName: name of the .tiff image
def readTif(fileName):
    dataset = gdal.Open(fileName)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    GdalImg_data = dataset.ReadAsArray(0, 0, width, height)
    return GdalImg_data

In [ ]:
# 4
# # This step was designed to convert the .tiff format into .jpg, etc. 
# # Besides, it can also be used to check the contents of the file and the image size
# # Utilization: Select all then uncomment
# n=14 #number of images at same area

# # Read all imgs in training data file 
# paths = glob.glob(os.path.join(sample1_Path,'*.tiff'))
# print(paths,sample1_Path)
# for i in paths:
# 	path= i 
# 	img= io.imread(path)

# 	#Subsection of the image
# 	#img=img[52:308,52:308]
# 	print (img.shape)
# 	readTif(path)
# 	#io.imsave(path+'.jpg',img)

# # Read all imgs in testing data file
# paths_test = glob.glob(os.path.join(sample2_Path,'*.tiff'))
# print(paths,sample2_Path)
# for j in paths:
# 	path= j 
# 	img= io.imread(path)
	
# 	#Subsection of the image
# 	#img=img[52:308,52:308]
# 	print (img.shape)
# 	readTif(path)
# 	# Test_Img[k].append(readTif(path))
# 	#io.imsave(path+'.jpg',img)


In [ ]:
# 5
# Read the 13 imgs with sepcific order
key = ["Elevation Local Mean","Elevation Local Maximum","Elevation Local Minimum","Elevation Local Std","Elevation Local Count",
       "Reflectance Local Mean","Reflectance Local Maximum","Reflectance Local Minimum","Reflectance Local Std",
       "Return Count Local Mean", "Return Count Local Maximum","Return Count Local Minimum","Return Count Local Std"];

In [ ]:
# 6
# Create Training data ## 创建训练数据
def create_train_data(sample1_Path):
  i = 0
  print('-'*30)
  print('Creating training images...')
  print('-'*30)
  imgs = glob.glob(os.path.join(sample1_Path,'*.tiff'))
  # imgs.sort(key=lambda x: int(x.split('/')[3][:-4])) # the order was defined according to "key", might be changed later
  print(len(imgs))
  # print(imgs)

  imgdatas = np.ndarray((len(imgs)-1,544,544,1), dtype=np.uint8) #############

  for key_item in key:
    for imgname in imgs:
      # midname = imgname[imgname.rindex("/")+1:]
      # if 'Landsat' in imgname:
      #   # i += 1
      #   # print('yes',imgname)
      #   continue
      # else:
      if key_item in imgname:
        img = image.load_img(imgname, color_mode='grayscale')
        print(i, imgname)
        img = image.img_to_array(img)
        imgdatas[i] = img
        i += 1
        continue

  print('loading done')
  # imgdatas = imgdatas.swapaxes(0, 1)
  # imgdatas = imgdatas.swapaxes(1, 2)
  # imgdatas = imgdatas.swapaxes(2,3)
  # img = img.swapaxes(2, 3)
  np.save(sample1_Path + '/imgs_train.npy', imgdatas)
  
  # np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
  print('Saving to .npy files done.')
  print(imgdatas.shape)

In [ ]:
# 7
# Create Testing data ## 创建训练数据
def create_test_data(sample1_Path):
  
  i = 0
  print('-'*30)
  print('Creating testing images...')
  print('-'*30)
  imgs = glob.glob(os.path.join(sample2_Path,'*.tiff'))
  # imgs.sort(key=lambda x: int(x.split('/')[3][:-4])) # the order was defined according to "key", might be changed later
  print(len(imgs))
  # print(imgs)

  imgdatas = np.ndarray((len(imgs)-1,544,544,1), dtype=np.uint8) #############

  for key_item in key:

    for imgname in imgs:
      # midname = imgname[imgname.rindex("/")+1:]
      # if 'Landsat' in imgname:
      #   # i += 1
      #   # print('yes')
      #   continue
      # else:
        ##########################
      if key_item in imgname:
        img = image.load_img(imgname, color_mode='grayscale')
        print(i,imgname)

        # print("*")

        img = image.img_to_array(img)
        imgdatas[i] = img

        continue

  print('loading done')
  # imgdatas = imgdatas.swapaxes(0, 1)
  # imgdatas = imgdatas.swapaxes(1, 2)
  # imgdatas = imgdatas.swapaxes(2,3)
  # img = img.swapaxes(2, 3)
  np.save(sample2_Path + '/imgs_test.npy', imgdatas)
  print('Saving to .npy files done.')
  print(imgdatas.shape)


In [ ]:
# 8
create_train_data(sample1_Path)
# print(imgdatas.shape)

In [ ]:
# 9
create_test_data(sample2_Path)
# print(imgdatas.shape)

In [ ]:
# 10
# Load training data ## 加载训练图片与mask
def load_train_data(self_path):
  '''
    return: the .npy data of training images, format supposed to be: (13,544,544,1)
  '''
  print('-'*30)
  print('load train images...')
  print('-'*30)

  imgs_train = np.load(self_path+"/imgs_train.npy")
  imgs_train = imgs_train.astype('float32')
  imgs_train /= 255
# denoising if necessiary
  # mean = imgs_train.mean(axis = 0)
  # imgs_train -= mean

  return imgs_train

In [ ]:
# 11
# Load testing data ## 加载训练图片与mask
def load_test_data(self_path):
    '''
    return: the .npy data of testing images, format supposed to be: (13,544,544,1)
  '''
  print('-'*30)
  print('load test images...')
  print('-'*30)
  imgs_test = np.load(self_path+"/imgs_test.npy")
  # imgs_mask_test = np.load(self.npy_path+"/imgs_mask_test.npy")
  imgs_test = imgs_test.astype('float32')
  # imgs_mask_test = imgs_mask_test.astype('float32')
  imgs_test /= 255
  # mean = imgs_test.mean(axis = 0)
  # imgs_test -= mean
  # imgs_mask_test /= 255
  # imgs_mask_test[imgs_mask_test > 0.5] = 1
  # imgs_mask_test[imgs_mask_test <= 0.5] = 0
  return imgs_test



In [ ]:
# 12
X_train = load_train_data(sample1_Path)
print(X_train.shape)

In [ ]:
# 13
X_test = load_test_data(sample2_Path)
print(X_test.shape)

In [ ]:
# 14
# Define convolutional block
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

In [ ]:
# 15
# Define Encoder block
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [ ]:
# 16
# Define Decoder block
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [ ]:
# 17
# Build UNet-like Autoencoder Architecture
def build_unet(input_shape):
    inputs = Input(input_shape)
    n_classes = 1
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  

    model = Model(inputs,outputs)
    return model

In [ ]:
# 18
# build and show the structure of the network
# Attention: need to change the input image size ########################################
input_img = shape=(544,544,1)### 1 band, grayscale
my_unet = build_unet(input_shape=(544,544,1))
print(my_unet.summary())

In [ ]:
# 19
# Compile the designed UNet-like AutoEncoder
my_unet.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
# 20
# Check the format of training and testing data
print(X_train.shape, X_test.shape)

In [ ]:
# 21
# Loss function designed part, in this case I chose binary crossentropy, but the reconstruction loss might be considered later.
# reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
# reconstruction_loss *= original_dim
# kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
# kl_loss = K.sum(kl_loss, axis=-1)
# kl_loss *= -0.5
# vae_loss = K.mean(reconstruction_loss + kl_loss)
# vae.add_loss(vae_loss)
# vae.compile(optimizer='adam')

In [ ]:
# 22
# X_train = X_train.astype('float32') 
print(X_train.max()) # make sure that the input pixel ranged within (0,1)
# X_test = X_test.astype('float32') / 255.

In [ ]:
# 23
# Train the model
# Attention: it may take serveral minuten, for Colab the batch_size cannot be setted too large, and the epochs could be larger, such as 20, 50, ...) 
# Question might affect the result: the setup of the training parameters. 
history = my_unet.fit(X_train, X_train,
                epochs=50,
                batch_size=5, verbose=1,shuffle=True,
                validation_data=(X_test,X_test))####### test data


In [ ]:
# 24
# list all data in model
print(history.history.keys())

# summarize model for loss, visulization the loss and val_loss value
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# 25
# summarize model for accuracy and val_accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
###################### Training Part is done ##########################

In [ ]:
# 26
# Save the model to Google Cloud Drive
PATH = '/content/drive/MyDrive/TUM/tran_Test-Equal-To-Train.pth'
torch.save(my_unet, PATH)


In [ ]:
# 27
# Reload the above trained model 
PATH = '/content/drive/MyDrive/TUM/tran_Test-Equal-To-Train.pth'
my_unet = torch.load(PATH)


In [ ]:
# 28
# Predict on another dataset 
# Attention: it may take several minutes, if you use Google Colab Notebook pls make sure your internet connection is stable.  !
decoded_imgs = my_unet.predict(X_train)



In [ ]:
# 29
# Checkt the format of the predicted data 
print(decoded_imgs.shape)

In [ ]:
# 30
# Show the initial and predicted results
n = 13

plt.figure(figsize=(20,4))
for i in range(0, X_train.shape[0]): #n
    # Display original
    ax = plt.subplot(2, n, 1+i)
    ax.imshow(X_train[i,:,:,0],cmap = 'gray')
    # cv2.imwrite(str(i)+".png",image[i,:,:])
    # plt.show()
    # ax.imshow(img[:,:,0], cmap='gray')

    # plt.imshow(X_test.reshape(28, 28))
    # plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i+1 + n)
    plt.imshow(decoded_imgs[i,:,:,0],cmap = 'gray')
    # plt.imshow(decoded_imgs[i].reshape(28, 28))
    # plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()


In [ ]:
############# Test Code is done here ##############################

In [ ]:
############# Below is the structure of the complete UNet architecture (Not an Autoencoder but for Classification) ##############################

In [ ]:

import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt


from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
import cv2

class automaticmaplabelling():
    def __init__(self,modelPath,full_chq,imagePath,width,height,channels):
        print (modelPath)
        print(imagePath)
        print(width)
        print(height)
        print(channels)
        self.modelPath=modelPath
        self.full_chq=full_chq
        self.imagePath=imagePath
        self.IMG_WIDTH=width
        self.IMG_HEIGHT=height
        self.IMG_CHANNELS=channels
        self.model = self.U_net()
        
    def mean_iou(self,y_true, y_pred):
        prec = []
        for t in np.arange(0.5, 1.0, 0.05):
            y_pred_ = tf.to_int32(y_pred > t)
            score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
            K.get_session().run(tf.local_variables_initializer())
            with tf.control_dependencies([up_opt]):
                score = tf.identity(score)
            prec.append(score)
        return K.mean(K.stack(prec), axis=0)

    def U_net(self):
        # Build U-Net model
        inputs = Input((self.IMG_HEIGHT, self.IMG_WIDTH, self.IMG_CHANNELS))
        s = Lambda(lambda x: x / 255) (inputs)

        c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
        c1 = Dropout(0.1) (c1)
        c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
        p1 = MaxPooling2D((2, 2)) (c1)

        c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
        c2 = Dropout(0.1) (c2)
        c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
        p2 = MaxPooling2D((2, 2)) (c2)

        c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
        c3 = Dropout(0.2) (c3)
        c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
        p3 = MaxPooling2D((2, 2)) (c3)

        c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
        c4 = Dropout(0.2) (c4)
        c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
        p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

        c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
        c5 = Dropout(0.3) (c5)
        c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

        u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
        u6 = concatenate([u6, c4])
        c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
        c6 = Dropout(0.2) (c6)
        c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

        u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
        u7 = concatenate([u7, c3])
        c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
        c7 = Dropout(0.2) (c7)
        c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

        u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
        u8 = concatenate([u8, c2])
        c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
        c8 = Dropout(0.1) (c8)
        c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

        u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
        u9 = concatenate([u9, c1], axis=3)
        c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
        c9 = Dropout(0.1) (c9)
        c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

        outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

        model = Model(inputs=[inputs], outputs=[outputs])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[self.mean_iou])
        model.load_weights(self.modelPath)
        model.summary()
        return model

    def prediction(self):
        img=cv2.imread(self.imagePath,0)
        img=np.expand_dims(img,axis=-1)
        x_test= np.zeros((1, self.IMG_HEIGHT, self.IMG_WIDTH, self.IMG_CHANNELS), dtype=np.uint8)
        #testimg=resize(img,(self.IMG_HEIGHT,self.IMG_WIDTH),mode='constant',preserve_range=True)
        x_test[0]=img
        preds_test= self.model.predict(x_test, verbose=1)
        
        preds_test = (preds_test > 0.5).astype(np.uint8)
        mask=preds_test[0]
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i][j] == 1:
                    mask[i][j] = 255
                else:
                    mask[i][j] = 0
        merged_image = cv2.merge((mask,mask,mask))
        contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for each_contour in contours:
            x,y,w,h = cv2.boundingRect(each_contour)

            cv2.rectangle(merged_image,(x,y),(x+w,y+h),(0,0,255),4)

            print (x,y,w,h)
            cv2.imshow("merged_image",merged_image)
            cv2.waitKey(0)
        
        cv2.imwrite("mask.png",mask)

        return x_test[0],mask

def main():
    test_image_name = "Lidar-NYC LiDAR Return Count Local Mean[count_mean]-01_01_2017T00_00_00.tiff"
    automaticmaplabellingobj= automaticmaplabelling('model-dsbowl2018-1.h5',True,test_image_name,128,128,3)
    testimg,mask = automaticmaplabellingobj.prediction()
    print('Showing images..')
    cv2.imshow('img',testimg)
    dim = (544, 544)
    resized = cv2.resize(mask, dim, interpolation = cv2.INTER_AREA)
    
    cv2.imshow('mask',mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite("resized_mask.png",resized)

if __name__ == "__main__":
    main()